<a href="https://colab.research.google.com/github/taka-onohara/google-colab/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh','avh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Y = rgdpna, K = rkna, L = emp * avh, alpha = 1 - labsh, A = (the rest)
data['Y'] = data['rgdpna']
data['K'] = data['rkna']
data['L'] = data['emp'] * data['avh']
data['alpha'] = 1 - data['labsh']
data['A'] = data['Y'] / ((data['K'] ** data['alpha']) * (data['L'] ** data['labsh']))
data['test'] = data['A'] * (data['K'] ** data['alpha']) * (data['L'] ** data['labsh'])
data.drop(data.columns[[2,3,4,5,6]], axis=1, inplace=True)

# print(data.head(10))

# y = Y/L, k = K/L, ln(y) = ln(A) + alpha * ln(k)
data['ln(y)'] = np.log(data['Y'] / data['L'])
data['ln(A)'] = np.log(data['A'])
data['ln(k)'] = np.log(data['K'] / data['L'])

data['test1'] = data['ln(A)'] + data['alpha'] * data['ln(k)']
# print(data.head(10))

data['growth rate'] = data['ln(y)'].diff() * 100
data['TFP growth'] = data['ln(A)'].diff() * 100 
data['capital deepening'] = data['alpha'] * (data['ln(k)'].diff() * 100 )

data['test2'] = data['TFP growth'] + data['capital deepening']


# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

data = data.dropna()

print(data.head(10))


       year countrycode             Y         K              L     alpha  \
5925   1995         ITA  2.142860e+06  0.672925   40889.500036  0.474591   
4315   1995         GBR  1.840006e+06  0.582493   44219.432146  0.455778   
6135   1995         JPN  4.156866e+06  0.687249  129334.225177  0.401236   
4175   1995         FRA  2.015066e+06  0.605657   37882.299026  0.379842   
3265   1995         DEU  3.107615e+06  0.657256   57879.831003  0.338852   
12155  1995         USA  1.144990e+07  0.519351  231660.254131  0.392623   
5926   1996         ITA  2.170005e+06  0.689952   41269.639548  0.472588   
4316   1996         GBR  1.885852e+06  0.604429   44644.481403  0.466012   
12156  1996         USA  1.188185e+07  0.539946  235886.358648  0.392884   
4176   1996         FRA  2.043539e+06  0.619433   38025.337150  0.376637   

                 A          test     ln(y)     ln(A)      ln(k)     test1  \
5925   9764.643078  2.142860e+06  3.959023  9.186523 -11.014750  3.959023   
4315   69